# 🎨 SETUP SUPERSET - Configuration des tables SQL

**Objectif** : Créer les tables Spark SQL pour que Superset puisse se connecter

**Auteurs** : Nejma MOUALHI | Brieuc OLIVIERI | Nicolas TAING

---

## 🎯 Ce que fait ce notebook

1. Créer des **tables externes Spark SQL** pointant vers les fichiers Parquet Gold
2. Réparer les partitions de la table `fait_consultation`
3. Vérifier que toutes les tables sont accessibles
4. Tester les requêtes SQL pour Superset

In [10]:
# Imports
from pyspark.sql import SparkSession

print("✅ Imports OK")

✅ Imports OK


In [11]:
# Configuration Spark
spark = SparkSession.builder \
    .appName("CHU_Superset_Setup") \
    .config("spark.sql.hive.convertMetastoreParquet", "false") \
    .enableHiveSupport() \
    .getOrCreate()

print(f"✅ Spark {spark.version} démarré avec support Hive")
print("✅ spark.sql.hive.convertMetastoreParquet = false (fix schéma)")

✅ Spark 3.5.0 démarré avec support Hive
✅ spark.sql.hive.convertMetastoreParquet = false (fix schéma)


---

## 📊 ÉTAPE 1 : Création des tables externes

Ces tables permettent à Spark SQL (et donc Superset) d'interroger les fichiers Parquet.

In [12]:
# Créer toutes les tables externes

# Définitions des tables (SANS partitionnement pour dim_temps)
tables_definitions = {
    "dim_temps": """
        CREATE EXTERNAL TABLE dim_temps (
            id_temps STRING,
            date_complete DATE,
            annee INT,
            mois INT,
            trimestre INT,
            jour_semaine STRING,
            nom_mois STRING,
            est_weekend BOOLEAN,
            numero_jour_semaine INT
        )
        STORED AS PARQUET
        LOCATION '/home/jovyan/data/gold/dim_temps'
    """,

    "dim_patient": """
        CREATE EXTERNAL TABLE dim_patient (
            id_patient STRING,
            nom_hash STRING,
            prenom_hash STRING,
            sexe STRING,
            age INT,
            date_naissance DATE,
            ville STRING,
            code_postal STRING,
            pays STRING,
            groupe_sanguin STRING
        )
        STORED AS PARQUET
        LOCATION '/home/jovyan/data/gold/dim_patient'
    """,

    "dim_diagnostic": """
        CREATE EXTERNAL TABLE dim_diagnostic (
            code_diag STRING,
            libelle STRING,
            categorie STRING
        )
        STORED AS PARQUET
        LOCATION '/home/jovyan/data/gold/dim_diagnostic'
    """,

    "dim_professionnel": """
        CREATE EXTERNAL TABLE dim_professionnel (
            code_specialite STRING,
            id_prof STRING,
            nom STRING,
            prenom STRING,
            nom_specialite STRING
        )
        STORED AS PARQUET
        LOCATION '/home/jovyan/data/gold/dim_professionnel'
    """,

    "dim_etablissement": """
        CREATE EXTERNAL TABLE dim_etablissement (
            finess STRING,
            siret STRING,
            nom STRING,
            ville STRING,
            code_postal STRING,
            telephone STRING,
            email STRING,
            code_departement STRING,
            num_departement STRING,
            libelle_departement STRING,
            libelle_region STRING,
            abv_region STRING
        )
        STORED AS PARQUET
        LOCATION '/home/jovyan/data/gold/dim_etablissement'
    """,

    "fait_consultation": """
        CREATE EXTERNAL TABLE fait_consultation (
            id_consultation STRING,
            id_patient STRING,
            id_prof STRING,
            code_diag STRING,
            id_mutuelle STRING,
            id_temps STRING,
            date_consultation DATE,
            heure_debut TIMESTAMP,
            heure_fin TIMESTAMP,
            motif STRING,
            jour INT
        )
        PARTITIONED BY (annee INT, mois INT)
        STORED AS PARQUET
        LOCATION '/home/jovyan/data/gold/fait_consultation'
    """
}

print("📦 Création des tables externes...\n")

for i, (table_name, create_sql) in enumerate(tables_definitions.items(), 1):
    try:
        # D'abord DROP (commande séparée) - FORCE pour supprimer le metastore
        try:
            spark.sql(f"DROP TABLE IF EXISTS {table_name}")
            print(f"  🗑️  {table_name} - ancien metastore supprimé")
        except Exception as drop_err:
            print(f"  ℹ️  {table_name} - pas de metastore existant")

        # Puis CREATE
        spark.sql(create_sql)
        print(f"  ✅ {i}/6 - {table_name} créée")
    except Exception as e:
        print(f"  ❌ Erreur table {i} ({table_name}): {e}")

print("\n✅ Toutes les tables créées !")

📦 Création des tables externes...

  🗑️  dim_temps - ancien metastore supprimé
  ✅ 1/6 - dim_temps créée
  🗑️  dim_patient - ancien metastore supprimé
  ✅ 2/6 - dim_patient créée
  🗑️  dim_diagnostic - ancien metastore supprimé
  ✅ 3/6 - dim_diagnostic créée
  🗑️  dim_professionnel - ancien metastore supprimé
  ✅ 4/6 - dim_professionnel créée
  🗑️  dim_etablissement - ancien metastore supprimé
  ✅ 5/6 - dim_etablissement créée
  🗑️  fait_consultation - ancien metastore supprimé
  ✅ 6/6 - fait_consultation créée

✅ Toutes les tables créées !


---

## 🔧 ÉTAPE 2 : Réparer les partitions

**Important** : Pour les tables partitionnées, Spark doit scanner le répertoire pour découvrir toutes les partitions.

In [13]:
# Réparer les partitions de fait_consultation
print("🔧 Réparation des partitions de fait_consultation...\n")

spark.sql("MSCK REPAIR TABLE fait_consultation")

print("✅ Partitions réparées !")

# Afficher les partitions découvertes
partitions = spark.sql("SHOW PARTITIONS fait_consultation")
print(f"\n📊 {partitions.count()} partitions trouvées")
partitions.show(20, truncate=False)

🔧 Réparation des partitions de fait_consultation...

✅ Partitions réparées !

📊 94 partitions trouvées
+------------------+
|partition         |
+------------------+
|annee=2015/mois=10|
|annee=2015/mois=11|
|annee=2015/mois=12|
|annee=2015/mois=6 |
|annee=2015/mois=7 |
|annee=2015/mois=8 |
|annee=2015/mois=9 |
|annee=2016/mois=1 |
|annee=2016/mois=10|
|annee=2016/mois=11|
|annee=2016/mois=12|
|annee=2016/mois=2 |
|annee=2016/mois=3 |
|annee=2016/mois=4 |
|annee=2016/mois=5 |
|annee=2016/mois=6 |
|annee=2016/mois=7 |
|annee=2016/mois=8 |
|annee=2016/mois=9 |
|annee=2017/mois=1 |
+------------------+
only showing top 20 rows



---

## ✅ ÉTAPE 3 : Vérification des tables

Compter les lignes de chaque table pour s'assurer qu'elles sont bien chargées.

In [14]:
# Lister toutes les tables
print("📋 Tables disponibles :\n")
spark.sql("SHOW TABLES").show(truncate=False)

📋 Tables disponibles :

+---------+-----------------+-----------+
|namespace|tableName        |isTemporary|
+---------+-----------------+-----------+
|default  |dim_diagnostic   |false      |
|default  |dim_etablissement|false      |
|default  |dim_patient      |false      |
|default  |dim_professionnel|false      |
|default  |dim_temps        |false      |
|default  |fait_consultation|false      |
+---------+-----------------+-----------+



In [15]:
# Compter les lignes de chaque table
print("📊 COMPTAGE DES LIGNES\n")
print("=" * 60)

tables = [
    "dim_temps",
    "dim_patient",
    "dim_diagnostic",
    "dim_professionnel",
    "dim_etablissement",
    "fait_consultation"
]

total = 0
for table in tables:
    try:
        count = spark.sql(f"SELECT COUNT(*) as cnt FROM {table}").collect()[0]['cnt']
        total += count
        print(f"  {table:25s} : {count:>10,} lignes")
    except Exception as e:
        print(f"  {table:25s} : ❌ Erreur - {e}")

print("=" * 60)
print(f"  {'TOTAL':25s} : {total:>10,} lignes\n")
print("✅ Toutes les tables sont accessibles !")

📊 COMPTAGE DES LIGNES

  dim_temps                 :      4,748 lignes
  dim_patient               :    100,000 lignes
  dim_diagnostic            :     15,490 lignes
  dim_professionnel         :  1,048,575 lignes
  dim_etablissement         :        200 lignes
  fait_consultation         :  1,027,157 lignes
  TOTAL                     :  2,196,170 lignes

✅ Toutes les tables sont accessibles !


---

## 🔍 ÉTAPE 4 : Tests des requêtes Superset

Tester les requêtes qui seront utilisées dans Superset.

In [16]:
# TEST 1 : Consultations par année
print("🔍 TEST 1 : Consultations par année\n")

# Lire directement les fichiers Parquet (bypass Hive)
df_temps = spark.read.parquet("/home/jovyan/data/gold/dim_temps")
df_consult = spark.read.parquet("/home/jovyan/data/gold/fait_consultation")

# Créer des vues temporaires
df_temps.createOrReplaceTempView("dim_temps_tmp")
df_consult.createOrReplaceTempView("fait_consultation_tmp")

query1 = """
SELECT
    t.annee,
    COUNT(*) as nb_consultations,
    COUNT(DISTINCT f.id_patient) as patients_uniques
FROM fait_consultation_tmp f
JOIN dim_temps_tmp t ON f.id_temps = t.id_temps
GROUP BY t.annee
ORDER BY t.annee
"""

spark.sql(query1).show()

🔍 TEST 1 : Consultations par année

+-----+----------------+----------------+
|annee|nb_consultations|patients_uniques|
+-----+----------------+----------------+
| 2015|           33896|           28581|
| 2016|          184308|           85272|
| 2017|          133403|           74201|
| 2018|          160373|           81075|
| 2019|           87497|           58635|
| 2020|          162778|           81612|
| 2021|          145883|           78593|
| 2022|          101991|           66042|
| 2023|           17028|           15772|
+-----+----------------+----------------+



In [17]:
# TEST 2 : Top diagnostics par catégorie CIM-10
print("🔍 TEST 2 : Top diagnostics par catégorie CIM-10\n")

# Lire diagnostic
df_diag = spark.read.parquet("/home/jovyan/data/gold/dim_diagnostic")
df_diag.createOrReplaceTempView("dim_diagnostic_tmp")

query2 = """
SELECT
    d.categorie,
    COUNT(*) as nb_consultations,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as pourcentage
FROM fait_consultation_tmp f
JOIN dim_diagnostic_tmp d ON f.code_diag = d.code_diag
WHERE d.categorie IS NOT NULL
GROUP BY d.categorie
ORDER BY nb_consultations DESC
LIMIT 10
"""

spark.sql(query2).show(truncate=False)

🔍 TEST 2 : Top diagnostics par catégorie CIM-10

+---------+----------------+-----------+
|categorie|nb_consultations|pourcentage|
+---------+----------------+-----------+
|M        |172661          |16.81      |
|O        |93887           |9.14       |
|S        |91287           |8.89       |
|T        |65015           |6.33       |
|V        |52376           |5.10       |
|Z        |50938           |4.96       |
|Q        |42209           |4.11       |
|D        |34813           |3.39       |
|K        |32718           |3.19       |
|C        |32433           |3.16       |
+---------+----------------+-----------+



In [23]:
# TEST 3 : Consultations par sexe et tranche d'âge
print("🔍 TEST 3 : Consultations par sexe et tranche d'âge\n")

# Lire patient
df_patient = spark.read.parquet("/home/jovyan/data/gold/dim_patient")
df_patient.createOrReplaceTempView("dim_patient_tmp")

query3 = """
SELECT
    p.sexe,
    CASE 
        WHEN p.age < 18 THEN '0-17 ans'
        WHEN p.age < 30 THEN '18-29 ans'
        WHEN p.age < 50 THEN '30-49 ans'
        WHEN p.age < 65 THEN '50-64 ans'
        ELSE '65+ ans'
    END as tranche_age,
    COUNT(*) as nb_consultations
FROM fait_consultation_tmp f
JOIN dim_patient_tmp p ON f.id_patient = p.id_patient
GROUP BY p.sexe, CASE 
    WHEN p.age < 18 THEN '0-17 ans'
    WHEN p.age < 30 THEN '18-29 ans'
    WHEN p.age < 50 THEN '30-49 ans'
    WHEN p.age < 65 THEN '50-64 ans'
    ELSE '65+ ans'
END
ORDER BY p.sexe, tranche_age
"""

spark.sql(query3).show()

🔍 TEST 3 : Consultations par sexe et tranche d'âge

+------+-----------+----------------+
|  sexe|tranche_age|nb_consultations|
+------+-----------+----------------+
|female|   0-17 ans|          109235|
|female|  18-29 ans|           74036|
|female|  30-49 ans|          120365|
|female|  50-64 ans|           90167|
|female|    65+ ans|          214966|
|  male|   0-17 ans|           75423|
|  male|  18-29 ans|           51094|
|  male|  30-49 ans|           82206|
|  male|  50-64 ans|           61354|
|  male|    65+ ans|          148311|
+------+-----------+----------------+



In [24]:
# TEST 4 : Top spécialités médicales
print("🔍 TEST 4 : Top 10 spécialités médicales\n")

# Lire professionnel
df_prof = spark.read.parquet("/home/jovyan/data/gold/dim_professionnel")
df_prof.createOrReplaceTempView("dim_professionnel_tmp")

query4 = """
SELECT
    prof.nom_specialite,
    COUNT(*) as nb_consultations,
    COUNT(DISTINCT f.id_patient) as patients_differents
FROM fait_consultation_tmp f
JOIN dim_professionnel_tmp prof ON f.id_prof = prof.id_prof
WHERE prof.nom_specialite IS NOT NULL
GROUP BY prof.nom_specialite
ORDER BY nb_consultations DESC
LIMIT 10
"""

spark.sql(query4).show(truncate=False)

🔍 TEST 4 : Top 10 spécialités médicales

+-----------------------------------+----------------+-------------------+
|nom_specialite                     |nb_consultations|patients_differents|
+-----------------------------------+----------------+-------------------+
|Medecine Generale                  |519780          |99246              |
|Psychiatrie                        |54336           |42345              |
|Anesthesie-reanimation             |48083           |38516              |
|Radio-diagnostic                   |36867           |31041              |
|Pediatrie                          |33738           |28868              |
|Cardiologie et maladies vasculaires|28751           |25132              |
|Ophtalmologie                      |24374           |21761              |
|Medecine du travail                |24156           |21549              |
|Gynecologie-obstetrique            |19063           |17424              |
|Chirurgie generale                 |16566           |15368

In [ ]:
# TEST 5 : Distribution géographique (avec départements enrichis)
print("🔍 TEST 5 : Distribution géographique par région\n")

# Lire établissement
df_etab = spark.read.parquet("/home/jovyan/data/gold/dim_etablissement")
df_etab.createOrReplaceTempView("dim_etablissement_tmp")

query5 = """
SELECT
    e.libelle_region as region,
    e.abv_region,
    COUNT(DISTINCT e.finess) as nb_etablissements,
    COUNT(DISTINCT e.ville) as nb_villes
FROM dim_etablissement_tmp e
WHERE e.libelle_region IS NOT NULL
GROUP BY e.libelle_region, e.abv_region
ORDER BY nb_etablissements DESC
"""

spark.sql(query5).show(truncate=False)

🔍 TEST 5 : Distribution géographique par région

+--------------------------+----------+-----------------+---------+
|region                    |abv_region|nb_etablissements|nb_villes|
+--------------------------+----------+-----------------+---------+
|Nouvelle-Aquitaine        |NAQ       |40               |37       |
|Auvergne-Rh�ne-Alpes      |ARA       |24               |24       |
|Grand Est                 |GES       |23               |22       |
|Normandie                 |NOR       |18               |18       |
|Centre-Val de Loire       |CVL       |15               |15       |
|Hauts-de-France           |HDF       |15               |15       |
|Occitanie                 |OCC       |13               |13       |
|Ile-de-France             |IDF       |11               |11       |
|Bourgogne-Franche-Comt�   |BFC       |11               |11       |
|Bretagne                  |BRE       |8                |6        |
|Pays de la Loire          |PDL       |8                |8        |

---

## 📝 ÉTAPE 5 : Informations de connexion pour Superset

Voici les paramètres à utiliser dans Superset.

In [ ]:
print("""
═══════════════════════════════════════════════════════════
📊 CONFIGURATION SUPERSET - CHU DATA LAKEHOUSE
═══════════════════════════════════════════════════════════

🌐 URL Superset : http://localhost:8088

🔐 Identifiants :
   Username : admin
   Password : admin

🔌 Connexion Database (Settings → Database Connections) :

   Database Type : Apache Spark SQL
   
   SQLAlchemy URI :
   ─────────────────────────────────────────────────────────
   hive://spark-master:10000/default
   ─────────────────────────────────────────────────────────
   
   OU (si PyHive installé) :
   ─────────────────────────────────────────────────────────
   hive://spark-master:10000/default?auth=NOSASL
   ─────────────────────────────────────────────────────────

   Display Name : CHU_Gold_Layer

📊 Tables disponibles (6) :
   • dim_temps              (4,748 lignes)
   • dim_patient            (100,000 lignes)
   • dim_diagnostic         (100 lignes)
   • dim_professionnel      (100,000 lignes)
   • dim_etablissement      (3,500 lignes)
   • fait_consultation      (1,027,157 lignes)

✅ NEXT STEPS :
   1. Ouvrir http://localhost:8088
   2. Login avec admin/admin
   3. Settings → Database Connections → + DATABASE
   4. Copier l'URI ci-dessus
   5. Test Connection → CONNECT
   6. Data → Datasets → Ajouter les 6 tables
   7. Créer des graphiques et dashboards !

═══════════════════════════════════════════════════════════
""")

---

## ✅ SETUP TERMINÉ

### 🎯 Ce qui a été fait :

1. ✅ 6 tables Spark SQL créées
2. ✅ Partitions de `fait_consultation` réparées
3. ✅ Toutes les tables vérifiées (1M+ lignes au total)
4. ✅ Requêtes de test exécutées avec succès
5. ✅ Thrift Server actif sur port 10000

### 📚 Documentation :

Voir **TUTORIEL_SUPERSET.md** pour le guide complet de configuration.

**🎓 Workflow complet validé** :
```
CSV/PostgreSQL → Bronze → Silver → Gold → Spark SQL → Superset Dashboards
```
